In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

IEEE-CIS Fraud Detection

In this exerciss, we aim to develop a fraud detection mechanism with reasonable accuracy to prevent the loss of assets for consumers. The data comes from Vesta's real-world e-commerce transactions and contains a wide range of features from device type to product features.

Data Description

Transaction Table

TransactionDT: timedelta from a given reference datetime (not an actual timestamp)

TransactionAMT: transaction payment amount in USD

ProductCD: product code, the product for each transaction
card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
addr: address
dist: distance

P_ and (R__) emaildomain: purchaser and recipient email domain

C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.

D1-D15: timedelta, such as days between previous transaction, etc.

M1-M9: match, such as names on card and address, etc.

Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

Categorical Features: ProductCD, card1 - card6, addr1, addr2, P_emaildomain, R_emaildomain, M1 - M9

Identity Table Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions. They're collected by Vesta’s fraud protection system and digital security partners. (The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

Categorical Features: DeviceType, DeviceInfo, id_12 - id_38

In [ ]:
import datetime
import re
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import xgboost as xgb

from sklearn.model_selection import cross_validate, GridSearchCV, validation_curve,RandomizedSearchCV
from sklearn.utils import resample
from sklearn.model_selection import KFold, StratifiedKFold,TimeSeriesSplit, GroupKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, \
    roc_auc_score, confusion_matrix, classification_report, plot_roc_curve
from sklearn.model_selection import train_test_split

Import and Append Training & Test Datasets

In [ ]:
train_identity= pd.read_csv("../input/ieee-fraud-detection/train_identity.csv")
train_transaction= pd.read_csv('../input/ieee-fraud-detection/train_transaction.csv') 
df_train=pd.merge(train_transaction, train_identity, on="TransactionID", how="left")

In [ ]:
test_identity= pd.read_csv('../input/ieee-fraud-detection/test_identity.csv')
test_transaction= pd.read_csv('../input/ieee-fraud-detection/test_transaction.csv')
df_test=pd.merge(test_transaction, test_identity, on="TransactionID", how="left")
df_test.head(20)

In [ ]:
fraud = df_train[df_train['isFraud'] == 1]
fraud

Data Exploration

In the data exploration section for this project, I want to achieve the following objectives:
- Find out if fraud rates have any relationship with the type of payment card used
- Find out if fraud is correlated at all with time

In [ ]:
fraud['card4'].value_counts()
type(fraud['card4'].value_counts())
df_card = fraud['card4'].value_counts().reset_index()
df_card.columns = ['Card Type', 'Count']
df_card

In [ ]:
df_train['card4'].value_counts()
type(df_train['card4'].value_counts())
df_card_train = df_train['card4'].value_counts().reset_index()
df_card_train.columns = ['Card Type', 'Count']
df_card_train

In [ ]:
df_card_join=pd.merge(df_card, df_card_train, on="Card Type", how="left")
df_card_join['Rate'] = df_card_join['Count_x']/df_card_join['Count_y']
df_card_join.head(20)

The above shows a tie between Visa and Mastercard at 3.4% of fraud detection rates, where Discover has detected 7.7% of the sample having fradulent activities. American Express has the lowest deteciton rate.

Assuming the rate of fraud acitivity is evenly distributed on the market, Visa, Mastercard, and American Express should really pick up on their fraud detection measures to match that of Discover.

In [ ]:
df_train['TransactionDT'].value_counts()
type(df_train['TransactionDT'].value_counts())
df_time_train = df_train['TransactionDT'].value_counts().reset_index()
df_time_train.columns = ['Date', 'Count']
df_time_train

In [ ]:
from matplotlib import pyplot
import matplotlib.pyplot as plt 
import numpy as np

x = df_time_train.Date
y = df_time_train.Count
pyplot.scatter(x, y)

In [ ]:
x = fraud.TransactionDT
y = fraud.TransactionAmt
pyplot.scatter(x, y)

Per the above two plots, the time where the highest count of fraud incidents occur also seems to have the highest fraud transaction amounts.

In [ ]:
df_train['TransactionAmt'].value_counts()
type(df_train['TransactionAmt'].value_counts())
df_tran_train = df_train['TransactionAmt'].value_counts().reset_index()
df_tran_train.columns = ['Amount', 'Count']
df_tran_train

In [ ]:
x = df_tran_train.Amount
y = df_tran_train.Count
pyplot.scatter(x, y)

To further investigate the amount defrauded, below step is to drop the last row as outlier.

In [ ]:
df_tran_train2 = df_tran_train.drop(8431)
df_tran_train2.sort_values(by=['Amount'])

In [ ]:
x = df_tran_train2.Amount
y = df_tran_train2.Count
pyplot.scatter(x, y)

As can be seen above, most fraudulent transactions are below $1000.

**Data Preparation**

Identify missing values

In [ ]:
def null_col_80(df_train,n = 74, thresh=80):
    df_train_80 = (df_train.isnull().sum()/df_train.shape[0])*100
    df_train_80 = df_train_80.reset_index()
    df_train_80.columns = ['Column Name','Null Rate']
    df_train_80 = df_train_80.sort_values(by=['Null Rate'],ascending=False).reset_index(drop=True)
    if n:
        return df_train_80.head(n)
    else:
        return df_train_80
null_col_80(df_train, thresh=80)

Drop Columns wth At Least 80% Missing Value

In [ ]:
null_col = df_train.drop(df_train.columns[[419, 420, 402, 403, 416, 421, 422, 418, 417, 16, 39, 413, 45, 46, 44, 398, 399, 38, 428, 405, 404, 41, 40, 425, 427, 429, 409, 198, 214, 196, 218, 197, 217, 213, 202, 212, 211, 210, 209, 205, 203, 204, 219, 205, 194, 216, 207, 208, 201, 200, 199, 215, 220, 221, 222, 206, 393, 389, 392, 391, 390, 394, 395, 380, 388, 381, 386, 385, 384, 383, 382, 378, 379, 387]], axis=1) 

Further reduce the number of id_XX and VXXX olumns, by identifying unique columns, through finding similarly behaving columns.

In [ ]:
nan_df= null_col.isna()
nan_group = {}
for col in null_col.columns:
    count = null_col[col].isnull().sum()
    try:
        nan_group[count].append(col)
    except:
        nan_group[count] = [col]
        
for k,v in nan_group.items():
    print(f'#####' * 4)
    print(f'NAN count = {k} percent: {(int(k)/null_col.shape[0])*100} %')
    print(v)

In [ ]:
corr=nan_group[460110]

def make_corr(corr):
    cols= ["TransactionDT"] + corr
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr)

Based on the above, we can start grouping columns and reduce columns that doesn't add much value.

In [ ]:
grps = [[217,218,219],[223,224,225],[226],[228,229,230],[231,232,233,235,236,237],[240,241],[242,243,244,246,247,248,249,252,253,254,257,258,260,261,262],[263,264,265],[266,267,268,269],[273,274,275],[276,277,278]]

def reduce_group(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group(grps)

In [ ]:
corr1=nan_group[314]

def make_corr(corr1):
    cols= ["TransactionDT"] + corr1
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr1)

In [ ]:
grps = [[95,96,97],[98],[99,100],[101,102,103,104,104,106],[107],[108,109,110,111,112,113,114,115,116],[117,118,119],[120,121,122],[123,124,125],[126,127,128],[129,130,131],[132,133,134],[135,136,137]]

def reduce_group1(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group1(grps)

In [ ]:
corr2=nan_group[12]

def make_corr(corr2):
    cols= ["TransactionDT"] + corr2
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr2)

In [ ]:
grps = [[279,280],[284],[285],[286],[287],[290,291,292],[293,294,295,297,298,299],[302,303,304],[305],[306,307,308],[309,310,311,312],[316,317,318],[319,320,321]]

def reduce_group2(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group2(grps)

In [ ]:
corr3=nan_group[449124]

def make_corr(corr3):
    cols= ["TransactionDT"] + corr3
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr3)

In [ ]:
grps = [[220],[221,222,227],[234],[238,239],[245],[250,251],[255,256,259],[270,271,272]]

def reduce_group3(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group3(grps)

In [ ]:
corr4=nan_group[450909]

def make_corr(corr4):
    cols= ["TransactionDT"] + corr4
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr4)

In [ ]:
grps = [[172,173,176],[177,178,179,181,182,183],[186,187,190,191,192,193,196,199],[202,203,204],[205,206,207],[211,212,213],[214,215,216]]

def reduce_group4(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group4(grps)

In [ ]:
corr5=nan_group[450721]

def make_corr(corr5):
    cols= ["TransactionDT"] + corr5
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr5)

In [ ]:
grps = [[169,170,171,174],[175,180],[184,185],[188,189,194,195,197,198],[200,201],[208,209,210]]

def reduce_group5(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group5(grps)

In [ ]:
corr6=nan_group[508595]

def make_corr(corr6):
    cols= ["TransactionDT"] + corr6
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr6)

In [ ]:
grps = [[138,139,141]]

def reduce_group6(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group6(grps)

In [ ]:
corr7=nan_group[89164]

def make_corr(corr7):
    cols= ["TransactionDT"] + corr7
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr7)

In [ ]:
grps = [[75,76],[77,78],[79,80,81],[82,83],[84,85],[86,87],[88],[89],[90,91],[92,93,94]]

def reduce_group7(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group7(grps)

In [ ]:
corr8=nan_group[77096]

def make_corr(corr8):
    cols= ["TransactionDT"] + corr8
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr8)

In [ ]:
grps = [[53,54],[55,56],[57,58,59,60],[61,62],[63,64],[65],[66,67],[68],[69,70],[71,72,73,74]]

def reduce_group8(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group8(grps)

In [ ]:
corr9=nan_group[168969]

def make_corr(corr9):
    cols= ["TransactionDT"] + corr9
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr9)

In [ ]:
grps = [[35,36],[37,38],[39,40],[41],[42,43],[44,45],[46,47],[48,49],[50,51,52]]

def reduce_group9(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group9(grps)

In [ ]:
corr10=nan_group[76073]

def make_corr(corr10):
    cols= ["TransactionDT"] + corr10
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr10)

In [ ]:
grps = [[12,13],[14],[15,16,17,18],[19,20],[21,22],[23,24],[25,26],[27,28],[29,30],[31,32,33,34]]

def reduce_group10(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group10(grps)

In [ ]:
corr11=nan_group[279287]

def make_corr(corr11):
    cols= ["TransactionDT"] + corr11
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr11)

In [ ]:
grps = [[1],[2,3,4,5,6,7,8,9],[10,11]]

def reduce_group11(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group11(grps)

In [ ]:
corr12=nan_group[1269]

def make_corr(corr12):
    cols= ["TransactionDT"] + corr12
    plt.figure(figsize=(15,15))
    sns.heatmap(null_col[cols].corr(), cmap="RdBu_r", annot=True, center=0.0)
    plt.show()
make_corr(corr12)

In [ ]:
grps = [[281,282,283,288,289],[296],[300,301],[313,314,315]]

def reduce_group12(grps, c="V"):
    reduce=[]
    for g in grps:
        mx=0; vx=g[0]
        for gg in g:
            n=null_col[c+ str(gg)].nunique()
            if n> mx:
                mx=n
                vx=gg
        reduce.append(vx)
    print(reduce)
    
reduce_group12(grps)

Below are the reduced V columns after the above reductions.

In [ ]:
V=  [218, 224, 226, 229, 232, 240, 258, 264, 267, 274, 277]
V+= [96, 98, 99, 102, 107, 111, 117, 120, 123, 127, 130, 133, 136]
V+= [280, 284, 285, 286, 287, 291, 294, 303, 305, 307, 310, 317, 320]
V+= [220, 221, 234, 238, 245, 250, 259, 271]
V+= [176, 178, 187, 203, 207, 212, 215]
V+= [171, 180, 185, 188, 201, 209]
V+= [139]
V+= [76, 78, 80, 82, 84, 86, 88, 89, 91, 92]
V+= [54, 56, 59, 62, 63, 65, 67, 68, 70, 72]
V+= [36, 37, 40, 41, 42, 44, 47, 48, 52]
V+= [13, 14, 17, 20, 22, 23, 26, 27, 30, 32]
V+= [1, 3, 11]
V+= [283, 296, 301, 314]

len(V)

**Data Cleaning Result**

Below is the final dataset for model building and testing.

In [ ]:
clean_data = null_col.drop(null_col.columns[null_col.columns.get_loc('V1'):null_col.columns.get_loc('V323')+1], axis=1)
clean_data

In [ ]:
for col in clean_data.columns:
    print(col)

**Model Building & Testing**

In [ ]:
result_test = clean_data.index[100000:]

Y_Train = clean_data["isFraud"]
Y_Test = Y_Train[result_test]
Y_Test

In [ ]:
Null_to_Zero = clean_data.fillna(0)
Null_to_Zero = Null_to_Zero.astype('string')

result_index = Null_to_Zero.index[:490540]
result_cols=Null_to_Zero.columns

from xgboost import XGBRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
model = XGBRegressor(n_estimators = 500, learning_rate = 0.05,max_depth=6)

X_Train=Null_to_Zero.loc[result_index,result_cols]
X_Test=Null_to_Zero.loc[result_index,result_cols]

le = LabelEncoder()

Y_master_Train=Y_Train[result_index]
cat_attribs = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_15', 'id_30', 'id_31', 'id_33', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), cat_attribs)], remainder='passthrough')
Null_to_Zero = Null_to_Zero.apply(lambda col: le.fit_transform(col.astype(str)), axis=0, result_type='expand')

encoder = full_pipeline.fit(X_Train)
X_Train = encoder.transform(X_Train)
X_Test = encoder.transform(X_Test)

model.fit(X_Train, Y_master_Train, verbose=False)

Y_pred = model.predict(X_Test)
Y_pred